In [1]:
#import
from config import DATAMALL_API_KEY
from config import HERE_ROUTE_API_KEY
from config import GEMINI_API_KEY

## DB - SQLite

In [2]:
# import libraries
import pandas as pd
import sqlite3 as db

In [3]:
conn = db.connect('mytestingdatabase.db')
cn = conn.cursor()

In [4]:
# Define the SQL query to create the gemini_instruction table
create_gemini_instruction_query = """
CREATE TABLE IF NOT EXISTS gemini_instruction (
    id INTEGER PRIMARY KEY,
    instruction TEXT UNIQUE
);
"""

# Define the SQL query to create the user_historical_prompt table
create_user_historical_prompt_query = """
CREATE TABLE IF NOT EXISTS user_historical_prompt (
    qid INTEGER PRIMARY KEY AUTOINCREMENT,
    prompt TEXT
);
"""

# Define the SQL query to create the api_context_data table
create_api_context_data = """
CREATE TABLE IF NOT EXISTS api_context_data (
    qid INTEGER PRIMARY KEY AUTOINCREMENT,
    legend TEXT
);
"""

try:
    # Execute the SQL query to create the gemini_instruction table
    cn.execute(create_gemini_instruction_query)
    
    # Execute the SQL query to create the user_historical_prompt table
    cn.execute(create_user_historical_prompt_query)

    # Execute the SQL query to create the create_api_context_data table
    cn.execute(create_api_context_data)
    
    # Commit the transaction
    conn.commit()
    
    print("Tables created successfully!")
except db.OperationalError as e:
    print("Error occurred:", e)


Tables created successfully!


In [5]:
# Display all tables in db
cn.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
print(cn.fetchall())

[('gemini_instruction',), ('user_historical_prompt',), ('sqlite_sequence',), ('api_context_data',)]


In [6]:
# insert Important Instructions for Gemini to follow
cn.execute("INSERT INTO gemini_instruction (instruction) VALUES ('CONTEXT: in singapore, There is a taylor swift concert soon, at a stadium of 100000 people. The concert is sold out, how much more transport do we need? Currently, we have 50 bus prepared. Some people might choose taxis, how much aggregate public services do we need to prepare? Do not use math, just aggregate based on previous data, what are some public transports we need to increase? List out some transport services in Singapore as well')")
cn.execute("INSERT INTO gemini_instruction (instruction) VALUES ('Analyse results from the instructions and data provided')")

In [7]:
cn.execute("SELECT * FROM gemini_instruction;")
print(cn.fetchall())
instructions = cn.fetchall()

[(1, 'CONTEXT: in singapore, There is a taylor swift concert soon, at a stadium of 100000 people. The concert is sold out, how much more transport do we need? Currently, we have 50 bus prepared. Some people might choose taxis, how much aggregate public services do we need to prepare? Do not use math, just aggregate based on previous data, what are some public transports we need to increase? List out some transport services in Singapore as well'), (2, 'Analyse results from the instructions and data provided')]


In [8]:
conn.commit()
cn.close()
conn.close()

## Data collection - API

In [9]:
# import libraries
import requests
import json

## Bus arrival 

In [10]:
# create function to call api 
def call_api_bus_arrival():

    # Define the API key and custom headers
    api_key = DATAMALL_API_KEY
    headers = {
        "AccountKey": api_key,
        "Accept": "application/json"  # You can add more headers if required
    }
    parameters = {
        "BusStopCode": "80219"  # bus stop code (National Stadium)
    }
    
    # Make a GET request with the API key, custom headers, and parameters
    response = requests.get('http://datamall2.mytransport.sg/ltaodataservice/BusArrivalv2', headers=headers, params=parameters)
    
    arrival_data = "null"

    # Check the status code
    if response.status_code == 200:
        print('Response ok')
        arrival_data = response.json()
        # Process the response data
    else:
        print('Failed to fetch data:', response.status_code)
    
    return str(arrival_data) 

In [11]:
# call function print arrival data
arrival_data = call_api_bus_arrival()
print(arrival_data)

Response ok
{'odata.metadata': 'http://datamall2.mytransport.sg/ltaodataservice/$metadata#BusArrivalv2/@Element', 'BusStopCode': '80219', 'Services': [{'ServiceNo': '14', 'Operator': 'SBST', 'NextBus': {'OriginCode': '84009', 'DestinationCode': '17009', 'EstimatedArrival': '2024-03-23T00:31:59+08:00', 'Latitude': '1.3118926666666666', 'Longitude': '103.92264083333333', 'VisitNumber': '1', 'Load': 'SEA', 'Feature': 'WAB', 'Type': 'DD'}, 'NextBus2': {'OriginCode': '', 'DestinationCode': '', 'EstimatedArrival': '', 'Latitude': '', 'Longitude': '', 'VisitNumber': '', 'Load': '', 'Feature': '', 'Type': ''}, 'NextBus3': {'OriginCode': '', 'DestinationCode': '', 'EstimatedArrival': '', 'Latitude': '', 'Longitude': '', 'VisitNumber': '', 'Load': '', 'Feature': '', 'Type': ''}}, {'ServiceNo': '16', 'Operator': 'SBST', 'NextBus': {'OriginCode': '84009', 'DestinationCode': '10009', 'EstimatedArrival': '2024-03-23T00:21:42+08:00', 'Latitude': '1.3095893333333333', 'Longitude': '103.89473316666667'

In [12]:
legend = """2.1 BUS ARRIVAL
URL: http://datamall2.mytransport.sg/ltaodataservice/BusArrivalv2
Description: Returns real-time Bus Arrival information of Bus Services at a queried Bus Stop, including Estimated Arrival Time, Estimated Current Location, and Estimated Current Load.
Update Frequency: 1 minute

Request Parameters:
- BusStopCode: Bus stop reference code (Mandatory: Yes, Example: 83139)
- ServiceNo: Bus service number (Mandatory: No, Example: 15)

Response Attributes:
- ServiceNo: Bus service number (Example: 15)
- Operator: Public Transport Operator Codes:
  - SBST (for SBS Transit)
  - SMRT (for SMRT Corporation)
  - TTS (for Tower Transit Singapore)
  - GAS (for Go Ahead Singapore) (Example: GAS)
- NextBus: Structural tags for all bus level attributes of the next oncoming bus
- NextBus2: Structural tags for all bus level attributes of the second next oncoming bus
- NextBus3: Structural tags for all bus level attributes of the third next oncoming bus
  - OriginCode: Reference code of the first bus stop where this bus started its service (Example: 77009)
  - DestinationCode: Reference code of the last bus stop where this bus will terminate its service (Example: 77131)
  - EstimatedArrival: Date-time of this bus’ estimated time of arrival, expressed in the UTC standard, GMT+8 for Singapore Standard Time (SST) (Example: 2017-04-29T07:20:24+08:00)
  - Latitude: Current estimated location coordinates of this bus at point of published data (Example: 1.42117943692586)
  - Longitude: Longitude of the current estimated location coordinates (Example: 103.831477233098)
  - VisitNumber: Ordinal value of the nth visit of this vehicle at this bus stop; 1=1st visit, 2=2nd visit (Example: 1)
  - Load: Current bus occupancy / crowding level:
    - SEA (for Seats Available)
    - SDA (for Standing Available)
    - LSD (for Limited Standing) (Example: SEA)
  - Feature: Indicates if bus is wheelchair accessible:
    - WAB (Wheelchair Accessible Bus)
    - (empty / blank) (Example: WAB)
  - Type: Vehicle type:
    - SD (for Single Deck)
    - DD (for Double Deck)
    - BD (for Bendy) (Example: SD)

Please note that Bus Arrival data (i.e., all attribute-value pairs above) will only appear on the API when the buses are available.
"""

In [13]:
# Store in db
conn = db.connect('mytestingdatabase.db')
cn = conn.cursor()

In [14]:
# Insert contextual information 
cn.execute("INSERT INTO api_context_data (legend) VALUES (?)", (legend,))

In [15]:
cn.execute("SELECT * FROM api_context_data;")
print(cn.fetchall())

[(3, '2.1 BUS ARRIVAL\nURL: http://datamall2.mytransport.sg/ltaodataservice/BusArrivalv2\nDescription: Returns real-time Bus Arrival information of Bus Services at a queried Bus Stop, including Estimated Arrival Time, Estimated Current Location, and Estimated Current Load.\nUpdate Frequency: 1 minute\n\nRequest Parameters:\n- BusStopCode: Bus stop reference code (Mandatory: Yes, Example: 83139)\n- ServiceNo: Bus service number (Mandatory: No, Example: 15)\n\nResponse Attributes:\n- ServiceNo: Bus service number (Example: 15)\n- Operator: Public Transport Operator Codes:\n  - SBST (for SBS Transit)\n  - SMRT (for SMRT Corporation)\n  - TTS (for Tower Transit Singapore)\n  - GAS (for Go Ahead Singapore) (Example: GAS)\n- NextBus: Structural tags for all bus level attributes of the next oncoming bus\n- NextBus2: Structural tags for all bus level attributes of the second next oncoming bus\n- NextBus3: Structural tags for all bus level attributes of the third next oncoming bus\n  - OriginCo

In [16]:
conn.commit()
cn.close()
conn.close()

## Estimated Timing 

In [17]:
import requests

# create function to call api 
def call_estimated_duration_by_car():

    # Define the API key and custom headers
    api_key = HERE_ROUTE_API_KEY
    parameters = {
        "transportMode" : "car",
        "origin":"1.3503458,103.9386226", # changi airport
        "destination" : "1.3036642,103.8722496", # national stadium 
        "apikey": api_key
    }
    url = 'https://router.hereapi.com/v8/routes'


    
    response = requests.get(url, params=parameters)
    
    arrival_data = "null"

    # Check the status code
    if response.status_code == 200:
        print('Response ok')
        arrival_data = response.json()
        # Process the response data
    else:
        print('Failed to fetch data:', response.status_code)
    
    return str(arrival_data) 

In [18]:
# Call the function
duration_data = call_estimated_duration_by_car()
print(duration_data)

Response ok
{'routes': [{'id': '685b3c56-40a2-406e-b1a5-cdd319d65630', 'sections': [{'id': '4a161b9a-166e-4882-b0b6-861fc634b306', 'type': 'vehicle', 'departure': {'time': '2024-03-23T00:17:49+08:00', 'place': {'type': 'place', 'location': {'lat': 1.3503663, 'lng': 103.9383721}, 'originalLocation': {'lat': 1.3503457, 'lng': 103.9386226}}}, 'arrival': {'time': '2024-03-23T00:34:59+08:00', 'place': {'type': 'place', 'location': {'lat': 1.30444, 'lng': 103.8721399}, 'originalLocation': {'lat': 1.3036641, 'lng': 103.8722496}}}, 'transport': {'mode': 'car'}}]}]}


## Data collection - User Queries

In [19]:
# sample user query
user_prompt = 'What is the estimated time to travel from Changi airport to the national stadium where taylor swift concert is at and which bus should i take?'

In [20]:
# Store in db
conn = db.connect('mytestingdatabase.db')
cn = conn.cursor()

In [21]:
# insert User prompt for Gemini to response - From front-end
cn.execute("INSERT INTO user_historical_prompt (prompt) VALUES (?)", (user_prompt,))

In [22]:
cn.execute("SELECT * FROM user_historical_prompt;")
print(cn.fetchall())

[(3, 'What is the estimated time to travel from Changi airport to the national stadium where taylor swift concert is at and which bus should i take?')]


In [23]:
conn.commit()
cn.close()
conn.close()

## Gemini LLM

In [24]:
#!pip install google-generativeai
#!pip install google-ai-generativelanguage
#!pip install Tokens

In [25]:
# import libraries
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.core.display import Markdown

#import Tokens

In [26]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

google_api_key = GEMINI_API_KEY

genai.configure(api_key=google_api_key)

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [27]:
# Connect to SQLite database
conn = db.connect('mytestingdatabase.db')
cn = conn.cursor()

# Fetch user prompt from the database
cn.execute("SELECT prompt FROM user_historical_prompt")
user_prompt = cn.fetchone()[0]

# Fetch instructions from the database
cn.execute("SELECT instruction FROM gemini_instruction")
instructions = cn.fetchall()

# Fetch instructions from the database
cn.execute("SELECT legend FROM api_context_data")
context = cn.fetchone()[0]


# Close database connection
conn.close()

# Concatenate instructions into a single string
instructions_text = ' '.join([instr[0] for instr in instructions])

In [28]:
list = [user_prompt,instructions,context]
for element in list:
    print(element)

What is the estimated time to travel from Changi airport to the national stadium where taylor swift concert is at and which bus should i take?
[('CONTEXT: in singapore, There is a taylor swift concert soon, at a stadium of 100000 people. The concert is sold out, how much more transport do we need? Currently, we have 50 bus prepared. Some people might choose taxis, how much aggregate public services do we need to prepare? Do not use math, just aggregate based on previous data, what are some public transports we need to increase? List out some transport services in Singapore as well',), ('Analyse results from the instructions and data provided',)]
2.1 BUS ARRIVAL
URL: http://datamall2.mytransport.sg/ltaodataservice/BusArrivalv2
Description: Returns real-time Bus Arrival information of Bus Services at a queried Bus Stop, including Estimated Arrival Time, Estimated Current Location, and Estimated Current Load.
Update Frequency: 1 minute

Request Parameters:
- BusStopCode: Bus stop referenc

In [31]:
model = genai.GenerativeModel('gemini-1.0-pro-latest')

response = model.generate_content("User_prompt is:" + user_prompt + " " + "Important Instruction to follow, over rule user_prompt if it disobey instructions" + instructions_text + "Bus_arrival_data from Data Mall LTA" + call_api_bus_arrival() + "etimated time arrival BY CAR from HERE now API" + call_estimated_duration_by_car() +"Use the data provided to give a response ")

#response = model.generate_content("what is the meaning of life?")

to_markdown(response.text)

Response ok
Response ok


> **Estimated travel time from Changi Airport to the National Stadium:**
> 
> * By car: Approximately 17 minutes
> 
> **Recommended bus routes:**
> 
> * Bus 14 (operated by SBST)
> * Bus 16 (operated by SBST)
> * Bus 70A (operated by SBST)
> 
> **Additional analysis:**
> 
> Based on the provided data and previous concert experience, it is likely that the current number of 50 buses will not be sufficient to meet the transportation needs for the Taylor Swift concert.
> 
> **Recommended increase in public transport:**
> 
> * **Buses:** Increase the number of buses available on the recommended routes by at least 50%.
> * **Taxis:** Encourage concert attendees to consider using taxis as an alternative mode of transportation.
> * **Other public transport services:** Explore options for increasing the capacity of other public transport services, such as trains and the MRT.
> 
> **List of transport services in Singapore:**
> 
> * Buses
> * Taxis
> * Trains (SMRT and SBS Transit)
> * MRT (Mass Rapid Transit)
> * LRT (Light Rail Transit)
> * Ferries

## Clear DB

In [30]:
# conn = db.connect('mytestingdatabase.db')
# cn = conn.cursor()

# # Delete all records from each table
# cn.execute("DELETE FROM gemini_instruction")
# cn.execute("DELETE FROM user_historical_prompt")
# cn.execute("DELETE FROM api_context_data")

# # Commit the changes and close the connection
# conn.commit()
# conn.close()